**[OptPess-LP](https://arxiv.org/pdf/2106.02684.pdf)**
---

今回は上記の論文について解説していきます。

---

CMDPについての論文なのですが、CMDPについての事前知識があると想定して進めます。



まず、この論文では２つのアルゴリズムについて説明しています。


---


１つ目は**OptPess-LP**です。

このアルゴリズムは制約違反しないような方策を事前知識として既知であると仮定して、Violationのregretを0にしています。

そして、コストに関してのregretは$\tilde{\mathcal{O}}\left(\frac{H^3}{\tau-c^0} \sqrt{|\mathcal{S}|^3|\mathcal{A}| K}\right)$を達成しています。

Hは1エピソードのステップ数、$\tau$は制約に関してのパラメータ、$c_0$は制約違反しない安全な方策についてのパラメータです。

このコストに関してのリグレットを[他の論文](https://arxiv.org/pdf/2003.02189.pdf)に比べると、H倍大きくなってしまっていますが、制約に関してのViolationは高確率で0を満たしてるので、安全なアルゴリズムを使いたいときはいいよねと主張しています。

---

問題設定
---

